In [1]:
# -*- coding:utf-8 -*-
import os
import random
import ssl
import torch.cuda
from sklearn.metrics import f1_score
from torch.optim import lr_scheduler
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import re
import numpy as np
import pandas as pd
import argparse
from utils import *
from model import *
import warnings
warnings.filterwarnings('ignore')
random.seed(666)
np.random.seed(666)
torch.manual_seed(666)
torch.cuda.manual_seed_all(666)
# 配置
ssl._create_default_https_context = ssl._create_unverified_context
train_path="./train/"
train_label='./train.csv'


def train(n_f,tra_da,val_da,cfg):
    tra_data=HumanDataset(tra_da)
    tra_loader=DataLoader(tra_data,batch_size=cfg['bs'], shuffle=True, drop_last=True) # pin_memory=True,

    val_data=HumanDataset(val_da)
    val_loader=DataLoader(val_data,batch_size=cfg['bs'],shuffle=False,drop_last=True)

    get_loss = nn.BCELoss()
    if torch.cuda.is_available():
        get_loss=get_loss.cuda()

    model = cfg['model'](cfg)

    print(model)
    main_params = list(map(id, model.last_linear.parameters()))
    main_params += list(map(id, model.bn1.parameters()))
    main_params += list(map(id, model.conv1.parameters()))
    base_params = filter(lambda p: id(p) not in main_params, model.parameters())  # 过滤序列，过滤掉不符合条件的元素，返回由符合条件元素组成的新列表。
    opt = optim.Adam([
        {'params': base_params, 'lr': cfg['backbone_lr']},
        {'params': model.last_linear.parameters()},
        {'params': model.conv1.parameters()},
        {'params': model.bn1.parameters()},
    ], lr=cfg['lr'])

    if torch.cuda.is_available():
        model=model.cuda()

    scheduler = lr_scheduler.MultiStepLR(opt,cfg['milestones'],gamma=cfg['gamma'])

    f1_best = np.float('-inf')
    best_i = 0
    for i in range(3):
        # if i - best_i > 6 and i > 20:
        #     break
        print(f'epoch {i}')
        scheduler.step(i)
        model.train()
        opt.zero_grad()
        for curr_iter, (images, target) in enumerate(tra_loader):
            if torch.cuda.is_available():
                images = images.cuda()   # non_blocking=True
                target = torch.from_numpy(np.array(target)).float().cuda()  # non_blocking=True

            output = model(images)

            loss = get_loss(output, target)

            loss.backward()

            if (curr_iter + 1) % cfg['M'] == 0:
                opt.step()
                opt.zero_grad()

            if curr_iter % 200 == 0:
                print(loss)

        if True:  # evaulate
            y, pred = predict(model, val_loader)

            y = np.array(y)

            # for i in np.arange()
            label = (np.array(pred) > cfg['thres']).astype(int)

            f1 = f1_score(y, label, average='macro')
            if f1_best < f1:
                f1_best = f1
                best_i = i
                print(f'f1_score{f1}, improved save model.......')
                torch.save(model.state_dict(), f"./weights/{cfg['name']}_fold{n_f}.pkl")
            else:
                print(f'f1_score{f1}, best f1_score{f1_best}')



def data_deal(train_path,train_label,cf):
    # train_list=os.listdir(train_path)
    # train_list=[x for x in train_list if re.findall('.*_green',x)]

    dataset=pd.read_csv(train_label)
    dataset['Id']=train_path+dataset['Id']

    # dataset['suffix']='_green.png'

    target=np.zeros((len(dataset),28))
    for i, label in enumerate(dataset['Target']):
        label=[int(x) for x in label.split()]
        for l in label:
            target[i,l]=1
    folds=MultilabelStratifiedKFold(cfg['nfold'],shuffle=True,random_state=2022)
    for n_fold,(tr_idx,val_idx) in enumerate(folds.split(dataset['Id'],target)):
        print(n_fold,'------------------')
        if n_fold not in cf['fold']:
            continue
        tr_data=dataset.iloc[tr_idx]
        val_data=dataset.iloc[val_idx]

        train(n_fold,tr_data,val_data,cf)

    
if __name__=='__main__':
    parser=argparse.ArgumentParser(description="parameter configuration")
    parser.add_argument('-f', type=str, default="读取额外的参数")
    parser.parse_args()

    cfg={}

    cfg['nfold']=5
    cfg['fold']=[int(fold) for fold in range(5)]
    cfg['bs']=24
    cfg['model'] = xception_model
    cfg['gamma'] = 0.5
    cfg['milestones'] = [7, 13, 18, 23, 30]
    cfg['lr'] = 0.0004
    cfg['backbone_lr']=cfg['lr']/2
    cfg['name']='xception'
    cfg['thres'] = 0.3
    cfg['M'] = 1
    data_deal(train_path,train_label,cfg)








/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 ------------------
Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): SeparableConv2d

In [5]:
!pip install iterative-stratification


Looking in indexes: https://mirrors.aliyun.com/pypi/simple
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [8]:
!pip install pretrainedmodels

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 424.0 kB/s eta 0:00:00a 0:00:01
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=6d9ad69f16137738b3cb2c2be1260a1860471a38c1e5a364117d65bb59840337
  Stored in directory: /root/.cache/pip/wheels/8b/7c/22/a039c0c530a1a87a6c94185f5dd302a6cd9e9a6a2e66db5731
Successfully built pretrainedmodels
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.
